In [13]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [14]:
def read_image():
    img = cv2.imread(r"win.jpeg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255.0

    return img

In [15]:
def initialize_means(img, clusters):
    points = img.reshape((-1, img.shape[2]))
    m, n = points.shape

    means = np.zeros((clusters, n))

    for i in range(clusters):
        rand_indices = np.random.choice(m, size=10, replace=False)
        means[i] = np.mean(points[rand_indices], axis=0)

    return points, means

In [16]:
def distance(x1, y1, x2, y2):
    dist = np.square(x1-x2) + np.square(y1-y2)
    dist = np.sqrt(dist)

    return dist

In [17]:
def k_means(points, means, clusters):
  iterations = 10
  m,n = points.shape

  index = np.zeros(m)

  while iterations > 0:
    for j in range(m):

      min_dist = float('inf')
      temp = None

      for k in range(clusters):
        x1, y1 = points[j,0], points[j,1]
        x2, y2 = points[k,0], points[k,1]

        if distance(x1, y1, x2, y2) <= min_dist:
          min_dist = distance(x1, y1, x2, y2)
          temp = k
          index[j] = k

    for k in range(clusters):
      cluster_points = points[index == k]
      if len(cluster_points) > 0:
        means[k] = np.mean(cluster_points, axis=0)

    iterations -= 1

  return means, index

In [22]:
def compress_image(means, index, img, clusters):
  centroid = np.array(means)
  recovered = centroid[index.astype(int), :]

  recovered = recovered.reshape(img.shape)

  plt.imshow(recovered)
  plt.show()

  cv2.imwrite('compressed_' + str(clusters) + '_colors.png', recovered)
  return recovered

In [27]:

if __name__ == '__main__':
    img = read_image()

    clusters = 16  # Default number of clusters
    clusters = int(input('Enter the number of colors in the compressed image (default = 16): '))

    start_time = time.time()
    points, means = initialize_means(img, clusters)
    means, index = k_means(points, means, clusters)
    compress_image(means, index, img, clusters)
    end_time = time.time()
    computational_time = end_time - start_time

    # Calculate PSNR
    psnr_value = psnr(img, cv2.imread('compressed_' + str(clusters) + '_colors.png'))



    # Calculate Compression Ratio
    compressed_img = cv2.imread(r'D:\SEM 4\SIP\Project\K-means\compressed_64_colors.png')
    compressed_img = cv2.cvtColor(compressed_img, cv2.COLOR_BGR2RGB)
    compressed_img = compressed_img/255.0

    original_size = img.size
    compressed_size = compressed_img.size 
    compression_ratio = original_size / compressed_size

    print(f"Computational Time: {computational_time:.4f} seconds")
    print(f"PSNR: {psnr_value:.2f} dB")
    print(f"Compression Ratio: {compression_ratio:.2f}")



# Original image size = 44.9kb
# Compressed image size(2) = 1.79 kb
# Compressed image size(16) = 14.2 kb
# Compressed image size(64) = 23.8 kb